In [ ]:
# default_exp clock_plots

In [ ]:
#hide 
%run load_actiwatch_data.py
%run firsttime.py

from joblib import *
from nbdev.showdoc import *
# this is used to make Federal Holidays a nonschool day.  Note that we don't have any
# way to recognize school district unique holidays, like teacher work days of such 
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import matplotlib.pyplot as plt
%matplotlib inline

# Making Clock Plots

> Module takes timing data processed by 'analyze_by_person and outputs
clock plots with sleep and light information.

In order to make a clock plot, we need timing data with specific information we'd like to display on the plot. Therefore, it is recommended that the dataframes used are loaded after being processed using the **analyze_by_person** module included in the SALA library.

The sample plots here will be generated using the same example data found in the aforementioned package. Thus, we'll start by loading the parquet files written by the functions found there. Of particular interest are columns relating to light timings and sleep information.

In [ ]:
file_prefix = "example_output/"

In [ ]:
all_data = pd.read_parquet(file_prefix+'raw.parquet')
all_data.head()

,Off-Wrist Status,Activity,Marker,White Light,Red Light,Green Light,Blue Light,Sleep/Wake,Interval Status,UID,Group
DateTime,,,,,,,,,,,
2018-05-07 11:28:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:28:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:29:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:29:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:30:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1


In [ ]:
timing_data = pd.read_parquet(file_prefix+'timing.parquet', engine='fastparquet')
timing_data[
    ["Mins to LL from 4AM", "Mins to FL from 4AM",
     "Sunrise", "Sunset", "Sleep onset", "Sleep offset",
     "Sleep duration", "Sleep onset MSLM",
     "Sleep offset MSLM"]
    ].head()

,Mins to LL from 4AM,Mins to FL from 4AM,Sunrise,Sunset,Sleep onset,Sleep offset,Sleep duration,Sleep onset MSLM,Sleep offset MSLM
index,,,,,,,,,
0,1048.0,130.0,2018-05-21 20:07:06.463156+00:00,2018-05-22 11:27:22.047832+00:00,2018-05-22 21:33:30,2018-05-23 05:22:30,0 days 07:49:00,1293.5,322.5
1,1321.0,91.0,2018-05-15 20:13:49.329097+00:00,2018-05-16 11:20:10.452788+00:00,2018-05-17 02:13:30,2018-05-17 06:03:30,0 days 03:50:00,1573.5,363.5
2,1269.0,87.0,2018-05-13 20:16:19.055530+00:00,2018-05-14 11:17:39.760167+00:00,2018-05-14 21:37:30,2018-05-15 06:18:00,0 days 08:40:30,1297.5,378.0
3,999.0,108.0,2018-05-08 20:23:04.188246+00:00,2018-05-09 11:11:11.579525+00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,0 days 00:00:00,-25430400.0,-25431840.0
4,1072.0,135.0,2018-05-10 20:20:17.050530+00:00,2018-05-11 11:13:48.602636+00:00,2018-05-11 21:58:30,2018-05-12 05:05:00,0 days 07:06:30,1318.5,305.0


In [ ]:
#export
def map_mins_to_rads(d_series):
    """Maps a series of minutes to radians for plot making.
    
    #### Parameters
    
    d_series: pd.Series
        A series of minute data
  
    #### Returns
    
    (as a tuple) list of mins converted to radians, a converted median
    """
    median = d_series.median()
    p25 = d_series.quantile(0.25)
    p75 = d_series.quantile(0.75)
    
    return ([x/1440.0*2*np.pi for x in np.arange(p25,p75)], med/1440.0*2*np.pi)

In [ ]:
show_doc(map_mins_to_rads, title_level = 3)

<h4 id="map_mins_to_rads" class="doc_header"><code>map_mins_to_rads</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>map_mins_to_rads</code>(**`d_series`**)

Maps a series of minutes to radians for plot making.

#### Parameters

d_series: pd.Series
    A series of minute data

#### Returns

(as a tuple) list of mins converted to radians, a converted median

In [ ]:
#export
def time_print(mins: float):
    """Takes a time (minutes) and returns a printable format.
    
    #### Parameters
    
    mins: float
        A numeric value representing minutes.
    
    #### Returns
    
    A printable format
    """
    h = int(mints / 60.)
    m = int ( (mins - h * 60) )
    if h >= 24.0:
        h -= 24
    return '{:02d}:{:02d}'.format(h,m)

In [ ]:
show_doc(time_print, title_level = 3)

<h4 id="time_print" class="doc_header"><code>time_print</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>time_print</code>(**`mins`**:`float`)

Takes a time (minutes) and returns a printable format.

#### Parameters

mins: float
    A numeric value representing minutes.

#### Returns

A printable format

In [ ]:
#export
def make_clock_plots(timing_data, group_by:str, thresholds: list = [], figsize: tuple = (5, 10)):
    """Creates clock plots for a grouping within timing data.
    
    #### Parameters
    
    timing_data: pd.DataFrame
    
        Timing data to use. Must be processed by the analyze_by_person package within SALA
    group_by: str
        
        String to subset the timing data for
    thresholds: list
        
        List of light thresholds.
    figsize: tuple
    
        Desired size of outputted figure
    """
    
    sns.set_style("white")
    
    if not thresholds:
        thresholds = timing_data.Threshold.unique()
    
    g_cols = sns.color_palette("Set2", 7)
    box_rad = 0.3 / len(thresholds)
    mw = 2 * np.pi / 1440
    box_sep = 1.1
    
    ng = len(timing_data[group_by].unique())
    f = plt.figure(figsize = figsize)
    
    # gn = groupname, grp = group
    for gn, grp in enumerate(timing_data[group_by].unique()):
        ax = f.add_subplot(ng, 1, gn + 1, projection = "polar")
        tbg = timing_data[timing_data[group_by] == grp]
        sunrise = (tbg['Sunrise']*60).median()
        sunset = (tbg['Sunset']*60).median()
        
        dark = [x / 1440.0 * 2 * np.pi for x in np.arange(0, sunrise)]
        ax.bar(dark, np.ones_like(dark), width = 0.02, color = [0.42,0.42,0.42], linewidth = 0)
        dark = [x / 1440.0 * 2 * np.pi for x in np.arange(sunset, 1440)]
        ax.bar(dark, np.ones_like(dark), width = 0.02, color = [0.42, 0.42, 0.42], linewidth = 0)
        
        lli=[]
        lll=[]
        for i, thr in enumerate(thresholds):
            added = False
            tbgt = (
                timing_data[(timing_data[group_by] == grp) & (timing_data["Threshold"] == thr)]
            )
            onset = 4 * 60 + tbgt["Mins to FL from 4AM"]
            offset = 4 * 60 + tbgt["Mins to LL from 4AM"]
            onbox, onmed = map_mins_to_rads(onset)
            offbox, offmed = map_mins_to_rads(offset)
            ll = ax.bar(onbox, np.full(len(onbox), box_rad),
                       width = mw, bottom = 1.0 - (i + 1) * box_rad * box_sep,
                       color = g_cols[i], linewidth = 0, alpha = 1.0)
            _ = ax.bar(onmed, box_rad,
                      width = 0.02, bottom = 1.0 - (i + 1) * box_rad * box_sep,
                      color = [0.2, 0.2, 0.2], linewidth = 0)
            
            # attempting to deal with low threshold light onset w/o offset issues
            # sometimes present in small datasets
            if (len(ll) > 0):
                lli.append(ll)
                lll.append('{:3d}lx {}-{}'.format(thr, tprint(onset.median()), tprint(offset.median())) )
                added = True
            ll = ax.bar(offbox, np.full(len(offbox), box_rad),
                       width = mw, bottom = 1.0 - (i + 1) * box_rad * box_sep,
                       color = gcols[i], linewidth = 0, alpha = 1.0)
            _ = ax.bar(offmed, box_rad,
                    width = 0.02, bottom = 1.0 - (i + 1) * box_rad * box_sep, 
                    color = [0.2, 0.2, 0.2], linewidth = 0)
            if (len(ll) > 0) and (not added):
                lli.append(ll)
                lll.append('{}lx'.format(thr))
                
        offset = tbgt['Sleep offset MSLM']
        onset = tbgt['Sleep onset MSLM']
        offbox, offmed = map_mins_to_rads(offset)
        onbox, onmed = map_mins_to_rads(onset)
        p = ax.bar(offbox, np.full(len(offbox), 2 * box_rad), 
                  width = mw, bottom = 1.0 - (i + 3) * box_rad * box_sep, 
                  color = gcols[-2], linewidth = 0, alpha = 1.0)
        _ =ax.bar(offmed, 2 * box_rad, width = 0.02, 
                  bottom = 1.0 - (i + 3) * box_rad * box_sep, 
                  color=[0.2, 0.2, 0.2], linewidth=0)
        ll = ax.bar(onbox, np.full(len(onbox), 2 * box_rad), 
                  width = mw, bottom = 1.0 - (i + 3) * box_rad * box_sep, 
                  color = gcols[-2], linewidth = 0, alpha = 1.0)
        _ = ax.bar(onmed, 2 * box_rad, 
                  width = 0.02, bottom = 1.0 - (i + 3) * box_rad * box_sep, 
                  color = [0.2, 0.2, 0.2], linewidth = 0)
        lli.append(ll)
        lll.append('Sleep {}-{}'.format(tprint(onset.median()), tprint(offset.median())) )
        
        thetat = np.arange(0,6)*60
        thetalbl = ['00:00','04:00','08:00','12:00','16:00','20:00']
        ax.set_thetagrids(thetat, labels=thetalbl)
        ax.set_theta_direction(-1)
        ax.set_theta_offset(np.pi)
        
        # less radial ticks
        ax.set_rticks([])
        ax.set_rmax(1.0)
        ax.grid(False)
        
        # if gn + 1 == ng:
        ax.legend(lli,lll,loc=[1.01,0.01],prop={'family': 'monospace'})
        
        nuids = len(tbg.UID.unique())
        ndays = len(tbg.Date.unique())
        pdays = len(tbgt[['UID','Date']].drop_duplicates())
        title = "{}={}: {} subjects, {} dates, {} person-days".format(group_by,grp,nuids,ndays,pdays)
        ax.set_title(title, y = 1.02) #loc='center', ha='center', va='bottom')
        
    plt.subplots_adjust(wspace = 1.2)

In [ ]:
show_doc(make_clock_plots, title_level = 3)

<h4 id="make_clock_plots" class="doc_header"><code>make_clock_plots</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>make_clock_plots</code>(**`timing_data`**, **`group_by`**:`str`, **`thresholds`**:`list`=*`[]`*, **`figsize`**:`tuple`=*`(5, 10)`*)

Creates clock plots for a grouping within timing data.

#### Parameters

timing_data: pd.DataFrame

    Timing data to use. Must be processed by the analyze_by_person package within SALA
group_by: str
    
    String to subset the timing data for
thresholds: list
    
    List of light thresholds.
figsize: tuple

    Desired size of outputted figure

## TODO: Add examples of Clock Plots